In [ ]:
import tensorflow as tf
import numpy as np

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)


# Using loss functions and optimizers in Tensorflow

Tensorflow has a lot of inbuilt functions and classes that are very convenient to use.

In [tf.keras.losses](https://www.tensorflow.org/api_docs/python/tf/keras/losses/) we find loss functions and in [tf.keras.optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) we find optimizers, including the standard gradient descent.

## Mean Squared Error

In [18]:
BATCH_SIZE = 8
N_PREDICTED_FEATURES = 5

targets = tf.random.uniform((BATCH_SIZE, N_PREDICTED_FEATURES))
predictions = tf.random.uniform((BATCH_SIZE, N_PREDICTED_FEATURES))

mse_loss = tf.keras.losses.MeanSquaredError()
mean_squared_error = mse_loss(targets,predictions)
print("MSE error for the batch:\n", mean_squared_error.numpy(), "\n")

MSE error for the batch:
 0.22216006 



## Categorical CrossEntropy

In [15]:
labels = [[0,1,0],
         [0,0,1],
         [1,0,0],
         [1,0,0],
         [0,1,0]]

labels = tf.constant(labels, dtype=tf.float32)

predictions = tf.random.normal(labels.shape)

# turn network output into categorical probability distribution over the labels
predictions = tf.nn.softmax(predictions)


# calculate categorical crossentropy

CCE_loss = tf.keras.losses.CategoricalCrossentropy()
batch_loss = CCE_loss(labels, predictions)

print("CCE loss between predicted label probabilities and ground truth labels is:\n", batch_loss.numpy())

CCE loss between predicted label probabilities and ground truth labels is:
 0.8781191


## Binary CrossEntropy

In [19]:
labels = [1,0,0,1,0,0,1,0]
labels = tf.constant(labels, dtype = tf.float32)

predictions = tf.random.uniform(labels.shape)

BCE_loss = tf.keras.losses.BinaryCrossentropy()
batch_loss = BCE_loss(labels,predictions)

print("BCE loss between predicted label probabilities and ground truth labels is:\n", batch_loss.numpy())

BCE loss between predicted label probabilities and ground truth labels is:
 0.8894618


# Making use of optimizers in the train loop

- Optimizers take care of applying the computed gradients to update the parameters

In [20]:
# choose optimizer and loss

optimizer = tf.keras.optimizers.SGD(learning_rate=0.005,
                                   momentum=0)

loss_function = tf.keras.losses.MeanSquaredError()

# create data
X = tf.random.uniform((20,1), minval= 0, maxval = 10)
Y = X * np.pi


# a simple linear univariate model function without bias
def model(x, parameter):
    return x * parameter

# initialize parameter variable to a value far away from pi
parameter_estimate = tf.Variable(7.5, trainable=True, dtype=tf.float32)

print("before training:", parameter_estimate)

#iterate over epochs
for epoch in range(2):

    # iterate over training examples (no batch dimension, but loss_function can take that too)
    for x,y in zip(X,Y):
        
        # within GradientTape context manager, calculate loss between targets and prediction
        with tf.GradientTape() as tape:

            prediction = model(x, parameter_estimate)

            loss = loss_function(y, prediction)

        # outside of context manager, obtain gradients with respect to list of trainable variables
        gradients = tape.gradient(loss, [parameter_estimate])

        #apply gradients with optimizer
        optimizer.apply_gradients(zip(gradients, [parameter_estimate]))
        
print("after training: ", parameter_estimate)

before training: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=7.5>
after training:  <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.141593>
